In [140]:
import numpy as np
import math
import matplotlib.pyplot as plt 
import pandas as pd


In [141]:
dataset = pd.read_csv('Student_Performance.csv')
X = dataset.iloc[:,:-1].values 
y = dataset.iloc[:,-1].values
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers =[('encoder',OneHotEncoder(),[2])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
X = X[:,1:]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 0)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev = X_train_scaled[0:1000]
X_train_scaled = X_train_scaled[1000:8000]
X_train_scaled.shape
y_dev = y_train[0:1000]
y_train = y_train[1000:8000]
X_test_scaled = scaler .fit_transform(X_test)



In [142]:
def train_error(w,b,X,y):
    m = y.shape[0]
    error=0
    for i in range(m):
        error += math.pow((np.dot(w,X[i]) + b - y[i]),2)

    J = error/(2*m)
    return J

In [143]:
def cv_error(w,b,X,y):
    m = y.shape[0]
    error=0
    for i in range(m):
        error += math.pow((np.dot(w,X[i]) + b - y[i]),2)

    J = error/(2*m)
    return J

In [144]:
def test_error(w,b,X,y):
    m = y.shape[0]
    error=0
    for i in range(m):
        error += math.pow((np.dot(w,X[i]) + b - y[i]),2)

    J = error/(2*m)
    return J

In [145]:
def cost(w,b,X,y,lambda_):
    error = 0
    m = y.shape[0]
    for i in range(m):
        error += math.pow((np.dot(w,X[i]) + b - y[i]),2)

    J = error/(2*m)
    reg_cost = J + (lambda_/(2*m))*(np.sum(np.square(w)))
    return reg_cost
    
    

In [146]:
def gradient_descent(w,b,X,y,alpha,lambda_):
    wder,bder = derivatives(w,b,X,y,lambda_)
    temp_w = w - alpha*wder
    temp_b = b - alpha*bder
    w = temp_w
    b = temp_b
    return w,b

In [147]:
def derivatives(w,b,X,y,lambda_):
    w = np.array(w, dtype='float64')
    X = np.array(X, dtype='float64')
    y = np.array(y, dtype='float64')
    wder = np.zeros(X[1].shape[0],dtype = 'float64')
    bder=0
    m = y.shape[0]
    for i in range(m):
        wder += ((np.dot(w,X[i]) + b) - y[i])*X[i]
        bder += ((np.dot(w,X[i]) + b) - y[i])
    wder/=m
    wder = wder + ((lambda_/m)*w)
    bder/=m
    return wder,bder


In [196]:
def train(X,y):
    lambda_ = 0.001
    w = np.zeros(X[1].shape[0],dtype = 'float64')
    b = 0
    alpha = 0.00001
    J = cost(w,b,X,y,lambda_)
    c=0
    costs = []
    ws = []
    while J>0.00001:
        if(c==800):
            break
        if J<0.00001:
            break
        if c>100:
            alpha = 0.01
        if c>475:
            alpha = 0.1
        if c>500:
            alpha = 1
        w,b = gradient_descent(w,b,X,y,alpha,lambda_)
        print(f'The values of w and b after {c} epochs is {w} and {b} respectively')
        print()
        J = cost(w,b,X,y,lambda_)
        print(f'The cost after {c} epochs is {J}')
        print()
        costs.append(J)
        ws.append(w.copy())
        c+=1

    print()
    print(f'Values of w and b after training the model is {w} and {b} respectively')
    return w,b,costs,ws
        
            
        
        

In [197]:
w,b,costs,ws = train(X_train_scaled,y_train)

The values of w and b after 0 epochs is [0.00000363 0.00008    0.00017711 0.00000926 0.00000781] and 0.0005522585714285715 respectively

The cost after 0 epochs is 1711.2136368011722

The values of w and b after 1 epochs is [0.00000727 0.00016001 0.00035423 0.00001852 0.00001561] and 0.0011045116069527236 respectively

The cost after 1 epochs is 1711.1793457190313

The values of w and b after 2 epochs is [0.0000109  0.00024001 0.00053134 0.00002778 0.00002342] and 0.0016567591066279628 respectively

The cost after 2 epochs is 1711.145055324978

The values of w and b after 3 epochs is [0.00001453 0.00032001 0.00070844 0.00003704 0.00003123] and 0.00220900107050978 respectively

The cost after 3 epochs is 1711.1107656190154

The values of w and b after 4 epochs is [0.00001816 0.00040001 0.00088555 0.0000463  0.00003903] and 0.0027612374986536762 respectively

The cost after 4 epochs is 1711.076476601115

The values of w and b after 5 epochs is [0.0000218  0.00048001 0.00106266 0.00005556

In [198]:
tr_error = train_error(w,b,X_train_scaled,y_train)
tr_error


2.1003592557075197

In [199]:
dev_error = cv_error(w,b,X_dev,y_dev)
dev_error

1.956734546356441

In [200]:
test_err = test_error(w,b,X_test_scaled,y_test)
test_err

2.4091431120077673

In [201]:
def predictions(w,b,X,y):
    y_hat = []
    for i in range(X.shape[0]):
        pred = np.dot(w,X[i]) + b
        y_hat.append(pred)
    y_hat = np.array(y_hat)
    return y_hat
        

In [202]:
y_hat = predictions(w,b,X_test_scaled,y_test)


In [203]:
y_bar = np.sum(y_test)/y_test.shape[0]
y_bar

55.7015

In [204]:
def calc_Rcoeff(y_test,y_hat,y_bar):
    k = (np.sum(np.square(y_test - y_hat)))/(np.sum(np.square(y_test - y_bar)))
    R = 1 - k
    return R

In [205]:
R = calc_Rcoeff(y_test,y_hat,y_bar)
accuracy = R*100
print('The accuracy of the model is = ',accuracy,'%')

The accuracy of the model is =  98.59975220878404 %


In [206]:
np.set_printoptions(threshold=np.inf, suppress=True)

In [207]:
print(y_test)

[ 53.  50.  80.  24.  64.  47.  83.  56.  47.  61.  55.  52.  70.  86.
  47.  50.  27.  74.  65.  35.  51.  50.  51.  69.  56.  52.  38.  80.
  82.  52.  44.  93.  72.  29.  67.  22.  55.  45.  62.  45.  61.  58.
  41.  78.  52.  47.  86.  44.  31.  51.  38.  81.  59.  51.  76.  56.
  74.  56.  33.  66.  50.  83.  88.  60.  19.  62.  16.  64.  49.  87.
  51.  70.  75.  26.  81.  84.  57.  30.  58.  17.  48.  54.  69.  77.
  61.  49.  47.  46.  55.  36.  76.  75.  53.  71.  26.  35.  55.  30.
  47.  24.  51.  70.  55.  77.  39.  84.  32.  59.  54.  67.  77.  71.
  62.  68.  32.  59.  85.  57.  58.  30.  59.  30.  44.  44.  82.  56.
  58.  40.  60.  29.  79.  37.  84.  30.  47.  47.  78.  24.  43.  37.
  59.  55.  37.  59.  72.  73.  35.  50.  89.  75.  49.  75.  30.  70.
  27.  27.  75.  43.  42.  42.  77.  43.  22.  23.  61.  57.  49.  64.
  30.  26.  23.  86.  39.  79.  62.  73.  37.  31.  71.  47.  53.  81.
  80.  72.  82.  37.  65.  74.  44.  81.  29.  53.  76.  28.  44.  74.
  30. 

In [208]:
print(y_hat)

[49.78240293 52.20477189 78.34235287 24.14485596 67.4846031  46.66213112
 83.42283048 53.18206944 47.46390674 62.24836316 51.08566546 50.57792223
 70.83104561 87.5981485  43.62635361 49.83059686 27.29935615 72.06322528
 65.15699836 29.52402518 51.36999647 48.86107342 53.68202412 69.93611731
 54.95458265 47.42504568 39.05789549 79.27117939 79.6005629  49.37034887
 43.76310589 93.95584385 72.36466326 28.26420594 67.46092174 18.26877973
 53.05463559 40.32114769 61.65946717 43.36661449 61.98731846 56.00962886
 43.49396901 78.37465398 51.62173484 48.07639268 85.39301476 47.46390674
 26.49604831 53.13985473 33.80341375 81.60649335 58.08702512 48.42250021
 75.85593813 54.33651255 74.33170612 54.20602869 30.97829706 68.07464836
 51.94960056 82.34176601 89.15505014 61.34404921 19.88318162 59.4841918
 15.9222466  64.29620469 46.45116666 87.88522589 49.0285176  72.93333738
 75.0713488  23.64451832 79.33019744 78.58124843 61.3452778  29.92639228
 54.51335448 15.41450336 44.71399013 52.95331142 69.